# Initializing the SysML v2 API

In [1]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json
from datetime import datetime

#host = "<specify protocol://host:port of the server that is a provider of the SysML v2 REST/HTTP API"
host = "http://sysml2-dev.intercax.com:9000"

# Get projects

In [2]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    projects_data = list(map(lambda b: {'Project Name':b['name'], 'Project ID':b['@id']}, projects))
    df = pd.DataFrame.from_records(projects_data).style.hide(axis='index')
    # if len(projects_data) > 0:
    #     df = df.style.sort_values(by='Project Name')
    display(df)
else:
    pprint("Problem in fetching projects")

Project Name,Project ID
4a-Functional Allocation Thu Jun 16 17:18:42 EDT 2022,02235ab1-46b0-4e88-8781-24876dfb7797
13a-Model Containment Thu Jun 16 17:23:32 EDT 2022,0606afde-a508-4806-bcd1-92e41e23e60a
18-Use Case Thu Jun 16 17:27:05 EDT 2022,0b8386f4-97dd-419f-8607-92ca345d4356
14c-Language Extensions Mon Jun 13 13:47:00 EDT 2022,0bac2595-df17-4edb-901b-373c7a92d729
10b-Trade-off Among Alternative Configurations Mon Jun 13 13:43:06 EDT 2022,115117fd-7e7b-454b-a7c6-d30e51ff910e
11a-View-Viewpoint Thu Jun 16 17:22:33 EDT 2022,11f9c415-0e30-438a-be15-cef64b480ac5
a1234 Mon Jul 11 12:05:47 EDT 2022,124c09f1-c639-4eec-964c-1b996dbed1f2
a1234 Mon Jul 11 12:05:53 EDT 2022,18da0870-4b9e-454d-9db6-9d9c1e292489
15_19-Materials with Properties Thu Jun 16 17:26:15 EDT 2022,1eb1a5b0-938b-460f-a610-82b784793d28
9-Verification-simplified Mon Jun 13 13:42:46 EDT 2022,202b22e1-9a4d-48de-8f7d-d84caa2ca96c


# Create a new project

In [3]:
timestamp = datetime.now()
project_name = f"Spacecraft project with branches and tags - {timestamp}"
project_data = {
  "@type":"Project",
  "name": project_name,
  "description": "Spacecraft project with multiple commits, branches, and tags"
}

project_post_url = f"{host}/projects" 

project_post_response = requests.post(project_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(project_data))

project_id = ""

if project_post_response.status_code == 200:
    project_response_json = project_post_response.json()
    pprint(project_response_json)
    project_id = project_response_json['@id']
    project_name = project_response_json['name']
else:
    pprint(f"Problem in creating a new Spacecraft project at {timestamp}")
    pprint(project_post_response)

{'@id': '25616f90-bcb8-46d9-a24c-079f2f99019d',
 '@type': 'Project',
 'defaultBranch': {'@id': 'f146cf01-0523-44f7-bb1a-6fdad0d21851'},
 'description': 'Spacecraft project with multiple commits, branches, and tags',
 'name': 'Spacecraft project with branches and tags - 2022-07-19 '
         '15:34:59.073337'}


# Get branches

In [4]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches of Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
main,f146cf01-0523-44f7-bb1a-6fdad0d21851,None


# Get tags

In [5]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    pprint(tags)
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags of Spacecraft project {project_id}")
    pprint(tags_response)

[]


# Create 1st Commit

In [6]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Spacecraft System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Payload System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Propulsion System"
      }
    }
  ]
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit1_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit1_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '022286b4-dcdf-496b-994c-c1d4a2748129',
 '@type': 'Commit',
 'owningProject': {'@id': '25616f90-bcb8-46d9-a24c-079f2f99019d'},
 'previousCommit': None,
 'timestamp': '2022-07-19T15:35:13.438917-04:00'}


In [7]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit1_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit1_id}.")
    pprint(element_get_response)

Element Name,Element ID
Payload System,d051e410-9ad2-4101-94ff-a4110e73ec89
Propulsion System,957d97ce-41e5-43c1-b772-45cac191565c
Spacecraft System,433f4040-ea4c-4ebc-b948-2b45bc8c4a5a


# Create 2nd Commit

In [8]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Avionics System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Power System"
      }
    }
  ],
  "previousCommit": {
      "@id": commit1_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit2_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit2_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '38643857-0090-4b81-89b7-b3411f117336',
 '@type': 'Commit',
 'owningProject': {'@id': '25616f90-bcb8-46d9-a24c-079f2f99019d'},
 'previousCommit': {'@id': '022286b4-dcdf-496b-994c-c1d4a2748129'},
 'timestamp': '2022-07-19T15:35:17.309787-04:00'}


# Get all elements after the 2nd commit

In [9]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}.")
    pprint(element_get_response)

Element Name,Element ID
Avionics System,4ff90e5b-2ac9-488c-b2d2-0d017802b121
Payload System,d051e410-9ad2-4101-94ff-a4110e73ec89
Power System,256a44f2-4bf8-4ce9-95a5-3a0af661aeb3
Propulsion System,957d97ce-41e5-43c1-b772-45cac191565c
Spacecraft System,433f4040-ea4c-4ebc-b948-2b45bc8c4a5a


# Create a tag

In [10]:
tag_body = {
    "@type": "Tag",
    "name": "Spacecraft Internal Release 0.1",
    "taggedCommit": {
      "@id": commit2_id
    }
}

tag_post_url = f"{host}/projects/{project_id}/tags" 

tag_post_response = requests.post(tag_post_url, 
                                  headers={"Content-Type": "application/json"}, 
                                  data=json.dumps(tag_body))

tag1_id = ""

if tag_post_response.status_code == 200:
    tag_response_json = tag_post_response.json()
    pprint(tag_response_json)
    tag1_id = tag_response_json['@id']
else:
    pprint(f"Problem in creating a new tag in Spacecraft project {project_id}.")
    pprint(tag_post_response)

{'@id': '758b401a-d6c9-4e9d-89d9-16a3fee3c049',
 '@type': 'Tag',
 'name': 'Spacecraft Internal Release 0.1',
 'owningProject': {'@id': '25616f90-bcb8-46d9-a24c-079f2f99019d'},
 'referencedCommit': {'@id': '38643857-0090-4b81-89b7-b3411f117336'},
 'taggedCommit': {'@id': '38643857-0090-4b81-89b7-b3411f117336'},
 'timestamp': '2022-07-19T15:35:21.841551-04:00'}


# Get Tags

In [11]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags in Spacecraft project {project_id}.")
    pprint(tag_post_response)

Tag Name,Tag ID,Tagged Commit
Spacecraft Internal Release 0.1,758b401a-d6c9-4e9d-89d9-16a3fee3c049,{'@id': '38643857-0090-4b81-89b7-b3411f117336'}


# Create a new branch at the second commit (also tagged as 0.1)

In [12]:
branch_body = {
  "@type": "Branch",
  "name": "develop",
  "head": {
      "@id": commit2_id
    }
}

branch_post_url = f"{host}/projects/{project_id}/branches" 

branch_post_response = requests.post(branch_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(branch_body))

branch_develop_id = ""

if branch_post_response.status_code == 200:
    branch_response_json = branch_post_response.json()
    pprint(branch_response_json)
    branch_develop_id = branch_response_json['@id']
else:
    pprint(f"Problem in creating a new branch in Spacecraft project {project_id}")
    pprint(branch_post_response)

{'@id': 'cb011176-6e8f-46e0-a314-741edae69f26',
 '@type': 'Branch',
 'head': {'@id': '38643857-0090-4b81-89b7-b3411f117336'},
 'name': 'develop',
 'owningProject': {'@id': '25616f90-bcb8-46d9-a24c-079f2f99019d'},
 'referencedCommit': {'@id': '38643857-0090-4b81-89b7-b3411f117336'},
 'timestamp': '2022-07-19T15:35:26.061576-04:00'}


# Get branches

In [13]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches in Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
develop,cb011176-6e8f-46e0-a314-741edae69f26,{'@id': '38643857-0090-4b81-89b7-b3411f117336'}
main,f146cf01-0523-44f7-bb1a-6fdad0d21851,{'@id': '38643857-0090-4b81-89b7-b3411f117336'}


# Create a new commit in the develop branch

In [14]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"GN & C System"
      }
    }
  ],
  "previousCommit": {
      "@id": commit2_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body),
                                      params={"branchId":branch_develop_id})

commit3_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit3_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '2705a620-834a-421a-a24b-208344aec910',
 '@type': 'Commit',
 'owningProject': {'@id': '25616f90-bcb8-46d9-a24c-079f2f99019d'},
 'previousCommit': {'@id': '38643857-0090-4b81-89b7-b3411f117336'},
 'timestamp': '2022-07-19T15:35:29.750462-04:00'}


# Get Branches

In [15]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches in Spacecraft project {project_id}")
    pprint(commit_post_response)

Branch Name,Branch ID,Ref Commit (head)
develop,cb011176-6e8f-46e0-a314-741edae69f26,{'@id': '2705a620-834a-421a-a24b-208344aec910'}
main,f146cf01-0523-44f7-bb1a-6fdad0d21851,{'@id': '38643857-0090-4b81-89b7-b3411f117336'}


# Get all elements as of the latest commit on main branch

In [16]:
pprint(f"Fetching elements at commit {commit2_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}")
    pprint(element_get_response)

'Fetching elements at commit 38643857-0090-4b81-89b7-b3411f117336'


Element Name,Element ID
Avionics System,4ff90e5b-2ac9-488c-b2d2-0d017802b121
Payload System,d051e410-9ad2-4101-94ff-a4110e73ec89
Power System,256a44f2-4bf8-4ce9-95a5-3a0af661aeb3
Propulsion System,957d97ce-41e5-43c1-b772-45cac191565c
Spacecraft System,433f4040-ea4c-4ebc-b948-2b45bc8c4a5a


# Get all elements as of the latest commit on develop branch

In [17]:
pprint(f"Fetching elements at commit {commit3_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit3_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit3_id}")
    pprint(element_get_response)

'Fetching elements at commit 2705a620-834a-421a-a24b-208344aec910'


Element Name,Element ID
Avionics System,4ff90e5b-2ac9-488c-b2d2-0d017802b121
GN & C System,83fa4826-7df8-4a8f-8227-180be69c79f6
Payload System,d051e410-9ad2-4101-94ff-a4110e73ec89
Power System,256a44f2-4bf8-4ce9-95a5-3a0af661aeb3
Propulsion System,957d97ce-41e5-43c1-b772-45cac191565c
Spacecraft System,433f4040-ea4c-4ebc-b948-2b45bc8c4a5a


# Create a tag at the last commit on develop branch

In [18]:
tag_body = {
    "@type": "Tag",
    "name": "Spacecraft Internal Release 0.2 build 1",
    "taggedCommit": {
      "@id": commit3_id
    }
}

tag_post_url = f"{host}/projects/{project_id}/tags" 

tag_post_response = requests.post(tag_post_url, 
                                  headers={"Content-Type": "application/json"}, 
                                  data=json.dumps(tag_body))

tag1_id = ""

if tag_post_response.status_code == 200:
    tag_response_json = tag_post_response.json()
    pprint(tag_response_json)
    tag1_id = tag_response_json['@id']
else:
    pprint(f"Problem in creating a new tag in Spacecraft project {project_id}.")
    pprint(tag_post_response)

{'@id': '8721d368-432f-4253-a827-46f01a0f906d',
 '@type': 'Tag',
 'name': 'Spacecraft Internal Release 0.2 build 1',
 'owningProject': {'@id': '25616f90-bcb8-46d9-a24c-079f2f99019d'},
 'referencedCommit': {'@id': '2705a620-834a-421a-a24b-208344aec910'},
 'taggedCommit': {'@id': '2705a620-834a-421a-a24b-208344aec910'},
 'timestamp': '2022-07-19T15:58:41.175306-04:00'}


# Get all tags

In [19]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags in Spacecraft project {project_id}.")
    pprint(tag_post_response)

Tag Name,Tag ID,Tagged Commit
Spacecraft Internal Release 0.1,758b401a-d6c9-4e9d-89d9-16a3fee3c049,{'@id': '38643857-0090-4b81-89b7-b3411f117336'}
Spacecraft Internal Release 0.2 build 1,8721d368-432f-4253-a827-46f01a0f906d,{'@id': '2705a620-834a-421a-a24b-208344aec910'}
